In [26]:
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_reviews_with_pagination(film_url, max_reviews=100, output_file='reviews_with_sentiments.csv', film_title=''):
    reviews_data = []
    current_page = 1

    while len(reviews_data) < max_reviews:
        page_url = f"{film_url}/reviews/ord/date/status/all/perpage/10/page/{current_page}/"
        driver.get(page_url)
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'response'))
            )
        except Exception:
            print(f"Нет отзывов на странице {current_page} или страница недоступна.")
            break

        review_elements = driver.find_elements(By.CLASS_NAME, 'response')
        for review_element in review_elements:
            try:
                review_text = review_element.find_element(By.XPATH, './/span').text.strip()
                review_class = review_element.get_attribute("class")
                
                if "good" in review_class:
                    sentiment = 1
                elif "bad" in review_class:
                    sentiment = -1
                elif "neutral" in review_class:
                    sentiment = 0
                else:
                    continue
                

                reviews_data.append({"Film": film_title, "Review": review_text, "Sentiment": sentiment})
                if len(reviews_data) >= max_reviews:
                    break

                pd.DataFrame([reviews_data[-1]]).to_csv(output_file, index=False, mode='a', encoding='utf-8', header=False)
                
            except Exception:
                continue 


        if len(review_elements) < 10:
            break
        
        current_page += 1

    return reviews_data

with open('movies.json', 'r', encoding='utf-8') as file:
    movies = json.load(file)

driver = webdriver.Chrome()

output_file = 'reviews_with_sentiments.csv'
df = pd.DataFrame(columns=["Film", "Review", "Sentiment"])
df.to_csv(output_file, index=False, mode='w', encoding='utf-8')

for movie in movies:
    title = movie['title']
    url = movie['url']
    print(f"Собираем отзывы для фильма: {title}")
    try:
        reviews = get_reviews_with_pagination(url, output_file=output_file, film_title=title)
        print(f"Сохранено {len(reviews)} отзывов для фильма: {title}")
    except Exception as e:
        print(f"Ошибка для фильма {title}: {e}")

driver.quit()

print(f"Все данные сохранены в файл {output_file}")


Собираем отзывы для фильма: 1+1
Нет отзывов на странице 1 или страница недоступна.
Сохранено 0 отзывов для фильма: 1+1
Собираем отзывы для фильма: Интерстеллар
Сохранено 100 отзывов для фильма: Интерстеллар
Собираем отзывы для фильма: Побег из Шоушенка
Сохранено 100 отзывов для фильма: Побег из Шоушенка
Собираем отзывы для фильма: Зеленая миля
Сохранено 100 отзывов для фильма: Зеленая миля
Собираем отзывы для фильма: Остров проклятых
Сохранено 100 отзывов для фильма: Остров проклятых
Собираем отзывы для фильма: Бойцовский клуб
Сохранено 100 отзывов для фильма: Бойцовский клуб
Собираем отзывы для фильма: Форрест Гамп
Сохранено 100 отзывов для фильма: Форрест Гамп
Собираем отзывы для фильма: Властелин колец: Возвращение короля
Сохранено 100 отзывов для фильма: Властелин колец: Возвращение короля
Собираем отзывы для фильма: Унесённые призраками
Сохранено 100 отзывов для фильма: Унесённые призраками
Собираем отзывы для фильма: Властелин колец: Братство кольца
Сохранено 100 отзывов для филь

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

service = Service(executable_path="D:\\chromedriver.exe")
driver = webdriver.Chrome(service=service)

try:
    base_url = "https://www.kinopoisk.ru/lists/movies/top250/?utm_referrer=yandex.ru&page="
    all_movies = [] 

    for page in range(1, 6):
        driver.get(f"{base_url}{page}")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//a[@class="base-movie-main-info_link__YwtP1"]')))
        movies = driver.find_elements(By.XPATH, '//a[@class="base-movie-main-info_link__YwtP1"]')
        
        for movie in movies:
            title = movie.find_element(By.XPATH, './/div[@class="desktop-list-main-info_mainTitle__8IBrD"]').text
            url = movie.get_attribute("href")
            all_movies.append({"title": title, "url": url})
        
        print(f"Собрано {len(movies)} фильмов с страницы {page}")

    with open("movies.json", "w", encoding="utf-8") as f:
        json.dump(all_movies, f, ensure_ascii=False, indent=4)

    print(f"Данные успешно записаны в файл movies.json")

finally:
    driver.quit()


Собрано 50 фильмов с страницы 1
Собрано 50 фильмов с страницы 2
Собрано 50 фильмов с страницы 3
Собрано 50 фильмов с страницы 4
Собрано 50 фильмов с страницы 5
Данные успешно записаны в файл movies.json
